<a href="https://colab.research.google.com/github/juanflorezVe/COMP9058_lab1/blob/master/r00184264_juanflorez_assg1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Copying vervatim the code from the assignment
import tensorflow as tf
print(tf.__version__)


2.4.1


In [2]:
import tensorflow as tf
import numpy as np

from keras.utils import np_utils

fashion_mnist = tf.keras.datasets.fashion_mnist

# load the training and test data    
(tr_x, tr_y), (te_x, te_y) = fashion_mnist.load_data()

# reshape the feature data
tr_x = tr_x.reshape(tr_x.shape[0], 784)
te_x = te_x.reshape(te_x.shape[0], 784)

# noramlise feature data
tr_x = tr_x / 255.0
te_x = te_x / 255.0

print( "Shape of training features ", tr_x.shape)
print( "Shape of test features ", te_x.shape)


# one hot encode the training labels and get the transpose
tr_y = np_utils.to_categorical(tr_y,10)
tr_y = tr_y.T
print ("Shape of training labels ", tr_y.shape)

# one hot encode the test labels and get the transpose
te_y = np_utils.to_categorical(te_y,10)
te_y = te_y.T
print ("Shape of testing labels ", te_y.shape)

4423680/4422102 [==============================] - 0s 0us/step
Shape of training features  (60000, 784)
Shape of test features  (10000, 784)
Shape of training labels  (10, 60000)
Shape of testing labels  (10, 10000)


In [45]:
# from lecture (XX, page 49) March 9th.... 
# We have a set of trainning of 60000 rows (instances)
# and 748 columns ( Features)
# Let's create a vector W with the number of weights (784)
W = tf.Variable(tf.random.normal([ tr_x.shape[0],1], mean=0.0, stddev=0.05))

# Reshape the label data so that it is a real     
# column vector
tr_y = tr_y.reshape(1,-1)
tr_y = tr_y.reshape(1,-1)
W.shape

TensorShape([60000, 1])

Question 1, section a. Question1_1_1


In [46]:
# Let's implement the final softmax layer L2

In [47]:
L2 = tf.Variable(tf.random.normal([10, 1], mean=0.0, stddev=0.05))

In [48]:
L2

<tf.Variable 'Variable:0' shape=(10, 1) dtype=float32, numpy=
array([[ 0.06723057],
       [-0.03407849],
       [ 0.00821829],
       [-0.06534814],
       [ 0.06222615],
       [-0.0542239 ],
       [ 0.05096988],
       [-0.01547042],
       [ 0.03598062],
       [ 0.03473904]], dtype=float32)>

In [49]:
#Let's insert 784 parameters to each neuron
# A = W.T * X + b
b = tf.Variable([0.])
A = tf.matmul(tf.transpose(W), tr_x) + b


In [50]:
A.shape

TensorShape([1, 784])

In [51]:
L2.shape


TensorShape([10, 1])

In [ ]:
#Review slide 33 of VectorizedLogisticRgression
